from google.colab import drive
drive.mount('/content/drive')

In [32]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [33]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
import itertools

In [34]:
np.random.seed(42)
tf.random.set_seed(42)

In [35]:
# Importing libraries
from keras import backend as K
from keras.models import Sequential
from keras.layers import LSTM, TimeDistributed, Conv1D, MaxPooling1D, Flatten, BatchNormalization
from keras.layers import Dense, Dropout, Bidirectional
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.regularizers import l2

In [36]:
"""### Load Data"""

# Data directory
DATADIR = '/content/drive/MyDrive/Colab Notebooks/UCI HAR Dataset'

# Activity labels
ACTIVITIES = {
    1: 'WALKING',
    2: 'WALKING_UPSTAIRS',
    3: 'WALKING_DOWNSTAIRS',
    4: 'SITTING',
    5: 'STANDING',
    6: 'LAYING'
}

# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [37]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'{DATADIR}/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).to_numpy()
        )

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [38]:
def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'{DATADIR}/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).to_numpy(), y.values


In [39]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test, y_train_raw, y_test_raw
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_train_raw = load_y('train')
    y_test, y_test_raw = load_y('test')

    return X_train, X_test, y_train, y_test, y_train_raw, y_test_raw


In [40]:
# Enhanced CSV data processing for external data
def process_csv_data(file_path, train_mean=None, train_std=None):
    """
    Process external CSV data for inference
    Returns normalized data in the correct shape for the model
    """
    # Load CSV data with actual columns
    df = pd.read_csv(file_path)

    # Check for required columns
    required_columns = ['gFx', 'gFy', 'gFz', 'wx', 'wy', 'wz', 'ax', 'ay', 'az']
    missing_columns = [col for col in required_columns if col not in df.columns]

    if missing_columns:
        raise ValueError(f"Missing required columns: {missing_columns}")

    # Map CSV columns to required features
    raw_data = np.hstack([
        df[['gFx', 'gFy', 'gFz']].values,  # Body acceleration
        df[['wx', 'wy', 'wz']].values,     # Gyroscope
        df[['ax', 'ay', 'az']].values      # Total acceleration
    ])

    # Normalize if mean and std are provided
    if train_mean is not None and train_std is not None:
        raw_data = (raw_data - train_mean) / train_std

    # Create windows of 128 timesteps
    windows = []
    for i in range(0, len(raw_data) - 127, 32):  # Stride of 32 for less overlap
        window = raw_data[i:i+128]
        if len(window) == 128:  # Ensure complete window
            windows.append(window)

    if not windows:
        raise ValueError("Not enough data to create windows")

    return np.array(windows)


In [41]:
# Post-processing for predictions
def post_process_predictions(predictions, threshold=0.5, smoothing_window=5):
    """
    Apply post-processing to raw predictions:
    1. Threshold probabilities
    2. Apply smoothing to reduce noise
    3. Return predicted class indices and labels
    """
    # Get predicted class indices
    pred_indices = np.argmax(predictions, axis=1)

    # Apply smoothing with a sliding window
    if len(pred_indices) >= smoothing_window:
        smoothed_indices = np.zeros_like(pred_indices)
        for i in range(len(pred_indices)):
            start = max(0, i - smoothing_window // 2)
            end = min(len(pred_indices), i + smoothing_window // 2 + 1)
            window = pred_indices[start:end]
            # Most common class in window
            values, counts = np.unique(window, return_counts=True)
            smoothed_indices[i] = values[np.argmax(counts)]
        pred_indices = smoothed_indices

    # Map indices to activity labels (add 1 because activities are 1-indexed)
    pred_labels = [ACTIVITIES[idx + 1] for idx in pred_indices]

    return pred_indices, pred_labels

In [42]:
# Visualization functions
def plot_confusion_matrix(cm, class_names, title='Confusion Matrix', cmap=None, figsize=(10, 8), normalize=False):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        fmt = '.2f'
        title += ' (Normalized)'
    else:
        fmt = 'd'

    if cmap is None:
        cmap = plt.cm.Blues

    plt.figure(figsize=figsize)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=16)
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45, fontsize=12)
    plt.yticks(tick_marks, class_names, fontsize=12)

    # Add text annotations to confusion matrix
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                horizontalalignment="center",
                color="white" if cm[i, j] > thresh else "black",
                fontsize=12)

    plt.tight_layout()
    plt.ylabel('True label', fontsize=14)
    plt.xlabel('Predicted label', fontsize=14)
    return plt

In [43]:
def plot_classification_metrics(y_true, y_pred, class_names, figsize=(12, 8)):
    """
    Plot precision, recall, and F1 scores from classification results
    """
    # Calculate metrics
    precision, recall, f1, support = precision_recall_fscore_support(y_true, y_pred)

    # Create figure with 3 subplots
    fig, axes = plt.subplots(1, 3, figsize=figsize)

    # Prepare data
    metrics = [precision, recall, f1]
    titles = ['Precision', 'Recall', 'F1 Score']

    # Create bar charts
    for i, (metric, title) in enumerate(zip(metrics, titles)):
        axes[i].bar(np.arange(len(class_names)), metric, color='skyblue')
        axes[i].set_title(title, fontsize=16)
        axes[i].set_xticks(np.arange(len(class_names)))
        axes[i].set_xticklabels(class_names, rotation=45, ha='right', fontsize=12)
        axes[i].set_ylim(0, 1.0)
        axes[i].set_xlabel('Activity', fontsize=14)
        axes[i].set_ylabel('Score', fontsize=14)

        # Add value labels
        for j, v in enumerate(metric):
            axes[i].text(j, v + 0.02, f'{v:.2f}', ha='center', fontsize=12)

    plt.tight_layout()
    return fig

In [44]:
def plot_combined_metrics(y_true, y_pred, class_names, figsize=(12, 8)):
    """
    Plot a combined view of precision, recall, and F1 scores
    """
    # Calculate metrics
    precision, recall, f1, support = precision_recall_fscore_support(y_true, y_pred)

    # Create figure
    plt.figure(figsize=figsize)

    # Set up positions for grouped bars
    x = np.arange(len(class_names))
    width = 0.25

    # Create bar groups
    plt.bar(x - width, precision, width, label='Precision', color='#5DA5DA')
    plt.bar(x, recall, width, label='Recall', color='#FAA43A')
    plt.bar(x + width, f1, width, label='F1 Score', color='#60BD68')

    # Add labels and titles
    plt.title('Classification Performance Metrics by Activity', fontsize=16)
    plt.xlabel('Activity', fontsize=14)
    plt.ylabel('Score', fontsize=14)
    plt.xticks(x, class_names, rotation=45, ha='right', fontsize=12)
    plt.yticks(fontsize=12)
    plt.ylim(0, 1.0)
    plt.legend(fontsize=12)

    plt.tight_layout()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    return plt

In [45]:
# Configure session settings
session_conf = tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

# Start a session
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)

# Initializing parameters
epochs = 50
batch_size = 16
n_hidden = 32

In [46]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [47]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test, Y_train_raw, Y_test_raw = load_data()

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)


<ipython-input-37-2fc0c20de1db>:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  return pd.read_csv(filename, delim_whitespace=True, header=None)
<ipython-input-37-2fc0c20de1db>:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  return pd.read_csv(filename, delim_whitespace=True, header=None)
<ipython-input-37-2fc0c20de1db>:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  return pd.read_csv(filename, delim_whitespace=True, header=None)
<ipython-input-37-2fc0c20de1db>:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  return pd.read_csv(filename, delim_whitespace=True, header=None)
<ipython-input-37-2fc0c2

In [48]:
print(f"Timesteps: {timesteps}")
print(f"Input dimensions: {input_dim}")
print(f"Number of training samples: {len(X_train)}")
print(f"Shape of training data: {X_train.shape}")



Timesteps: 128
Input dimensions: 9
Number of training samples: 7352
Shape of training data: (7352, 128, 9)


In [49]:
# Calculate and store normalization parameters for external data processing
train_mean = np.mean(X_train.reshape(-1, input_dim), axis=0)
train_std = np.std(X_train.reshape(-1, input_dim), axis=0)
np.save('train_normalization_params.npy', {'mean': train_mean, 'std': train_std})

In [50]:
"""## Defining LSTM Network model:"""

# Define callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

model_checkpoint = ModelCheckpoint(
    'best_lstm_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

# Add ReduceLROnPlateau callback
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-6,
    verbose=1
)

In [51]:
# Initializing the sequential model with improved architecture
model1 = Sequential()

# First LSTM layer with increased units (64) and return_sequences=True
model1.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=(timesteps, input_dim)))
model1.add(BatchNormalization())
model1.add(Dropout(0.5))  # Dropout after first LSTM layer

# Second LSTM layer
model1.add(Bidirectional(LSTM(n_hidden)))
model1.add(BatchNormalization())
model1.add(Dropout(0.5))

# Dense layer with ReLU activation and L2 regularization
model1.add(Dense(n_hidden, activation='relu', kernel_regularizer=l2(0.01)))
model1.add(BatchNormalization())
model1.add(Dropout(0.5))

# Output layer with softmax activation
model1.add(Dense(n_classes, activation='softmax'))
model1.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional_4 (Bidirectional)      │ (None, 128, 128)            │          37,888 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_9                │ (None, 128, 128)            │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 128, 128)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_5 (Bidirectional)      │ (None, 64)                  │          41,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_10               │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_11               │ (None, 32)                  │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 6)                   │             198 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 82,278 (321.40 KB)

 Trainable params: 81,830 (319.65 KB)

 Non-trainable params: 448 (1.75 KB)

In [ ]:
# Compiling the model with Adam optimizer and learning rate of 0.001
model1.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

# Training the model with early stopping, checkpoints, and reduce LR
lstm_history = model1.fit(
    X_train,
    Y_train,
    batch_size=batch_size,
    validation_data=(X_test, Y_test),
    epochs=epochs,
    callbacks=[early_stopping, model_checkpoint, reduce_lr]
)

Epoch 1/50
460/460 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.4415 - loss: 2.0052
Epoch 1: val_accuracy improved from -inf to 0.73295, saving model to best_lstm_model.keras
460/460 ━━━━━━━━━━━━━━━━━━━━ 20s 30ms/step - accuracy: 0.4417 - loss: 2.0042 - val_accuracy: 0.7329 - val_loss: 0.9707 - learning_rate: 0.0010
Epoch 2/50
458/460 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.6750 - loss: 1.0863
Epoch 2: val_accuracy improved from 0.73295 to 0.82423, saving model to best_lstm_model.keras
460/460 ━━━━━━━━━━━━━━━━━━━━ 19s 27ms/step - accuracy: 0.6752 - loss: 1.0856 - val_accuracy: 0.8242 - val_loss: 0.6632 - learning_rate: 0.0010
Epoch 3/50
460/460 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.7918 - loss: 0.7461
Epoch 3: val_accuracy improved from 0.82423 to 0.87682, saving model to best_lstm_model.keras
460/460 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - accuracy: 0.7919 - loss: 0.7459 - val_accuracy: 0.8768 - val_loss: 0.4381 - learning_rate: 0.0010
Epoch 4/50
460/460 ━━━━━━━━━━━━

In [ ]:
# Testing
score = model1.evaluate(X_test, Y_test)
print("Accuracy: ", score[1])
print("Loss: ", score[0])

In [ ]:
# Generate predictions for confusion matrix
y_pred = model1.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(Y_test, axis=1)

# Get activity names for visualization
activity_names = [ACTIVITIES[i+1] for i in range(n_classes)]

# Generate and display enhanced confusion matrix
cm = confusion_matrix(y_true_classes, y_pred_classes)
# Regular confusion matrix
plot_confusion_matrix(cm, activity_names, title='LSTM Confusion Matrix', figsize=(12, 10))
plt.savefig('lstm_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# Normalized confusion matrix
plot_confusion_matrix(cm, activity_names, title='LSTM Confusion Matrix', figsize=(12, 10), normalize=True)
plt.savefig('lstm_confusion_matrix_normalized.png', dpi=300, bbox_inches='tight')
plt.show()

# Display text classification report
print("\nClassification Report:")
print(classification_report(y_true_classes, y_pred_classes, target_names=activity_names))

# Plot graphical classification report
plot_classification_metrics(y_true_classes, y_pred_classes, activity_names)
plt.savefig('lstm_classification_metrics.png', dpi=300, bbox_inches='tight')
plt.show()

# Plot combined metrics
plot_combined_metrics(y_true_classes, y_pred_classes, activity_names)
plt.savefig('lstm_combined_metrics.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:

"""## Defining CNN-LSTM Network model:"""

"""## Defining CNN-LSTM Network model:"""

# Reshape data for CNN-LSTM model
n_steps, n_length = 4, 32
X_train_cnn = X_train.reshape((X_train.shape[0], n_steps, n_length, input_dim))
X_test_cnn = X_test.reshape((X_test.shape[0], n_steps, n_length, input_dim))

print(f"Reshaped data for CNN-LSTM: {X_train_cnn.shape}")

In [ ]:
# Define model with improved architecture
model2 = Sequential()

# First Conv1D layer with increased filters (128) and larger kernel size (5)
model2.add(TimeDistributed(Conv1D(filters=128, kernel_size=5, activation='relu'),
                         input_shape=(None, n_length, input_dim)))
model2.add(TimeDistributed(BatchNormalization()))
model2.add(TimeDistributed(Dropout(0.5)))

# Second Conv1D layer with 64 filters
model2.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
model2.add(TimeDistributed(BatchNormalization()))
model2.add(TimeDistributed(Dropout(0.5)))

model2.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model2.add(TimeDistributed(Dropout(0.5)))  # Dropout after MaxPooling

model2.add(TimeDistributed(Flatten()))
model2.add(BatchNormalization())

# First LSTM layer with return_sequences=True
model2.add(Bidirectional(LSTM(64, return_sequences=True)))
model2.add(BatchNormalization())
model2.add(Dropout(0.5))

# Second LSTM layer
model2.add(Bidirectional(LSTM(n_hidden)))
model2.add(BatchNormalization())
model2.add(Dropout(0.5))

# Dense layer with ReLU activation and L2 regularization
model2.add(Dense(n_hidden, activation='relu', kernel_regularizer=l2(0.01)))
model2.add(BatchNormalization())
model2.add(Dropout(0.5))

# Output layer with softmax activation
model2.add(Dense(n_classes, activation='softmax'))
model2.summary()

In [ ]:
# Compile model with Adam optimizer and learning rate of 0.001
model2.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

# Define additional callbacks for CNN-LSTM
cnn_lstm_checkpoint = ModelCheckpoint(
    'best_cnn_lstm_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

# Training the model with all callbacks
cnn_lstm_history = model2.fit(
    X_train_cnn,
    Y_train,
    batch_size=batch_size,
    validation_data=(X_test_cnn, Y_test),
    epochs=epochs,
    callbacks=[early_stopping, cnn_lstm_checkpoint, reduce_lr]
)

In [ ]:
# Testing
score = model2.evaluate(X_test_cnn, Y_test)
print("Accuracy: ", score[1])
print("Loss: ", score[0])

In [ ]:
# Generate predictions for confusion matrix
y_pred = model2.predict(X_test_cnn)
y_pred_classes = np.argmax(y_pred, axis=1)

# Generate and display enhanced confusion matrix for CNN-LSTM
cm = confusion_matrix(y_true_classes, y_pred_classes)

# Regular confusion matrix
plot_confusion_matrix(cm, activity_names, title='CNN-LSTM Confusion Matrix', figsize=(12, 10))
plt.savefig('cnn_lstm_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# Normalized confusion matrix
plot_confusion_matrix(cm, activity_names, title='CNN-LSTM Confusion Matrix', figsize=(12, 10), normalize=True)
plt.savefig('cnn_lstm_confusion_matrix_normalized.png', dpi=300, bbox_inches='tight')
plt.show()

# Display text classification report
print("\nClassification Report:")
print(classification_report(y_true_classes, y_pred_classes, target_names=activity_names))

# Plot graphical classification report
plot_classification_metrics(y_true_classes, y_pred_classes, activity_names)
plt.savefig('cnn_lstm_classification_metrics.png', dpi=300, bbox_inches='tight')
plt.show()

# Plot combined metrics
plot_combined_metrics(y_true_classes, y_pred_classes, activity_names)
plt.savefig('cnn_lstm_combined_metrics.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
def plot_model_history(history, model_name):
    """
    Plots the training and validation accuracy and loss curves for a given model.

    Args:
        history: The training history object returned by model.fit.
        model_name: The name of the model (e.g., "LSTM", "CNN-LSTM").
    """

    fig, axs = plt.subplots(2, 1, figsize=(12, 10))

    # Plot training & validation accuracy values
    axs[0].plot(history.history['accuracy'], linewidth=2)
    axs[0].plot(history.history['val_accuracy'], linewidth=2)
    axs[0].set_title(f'{model_name} Model Accuracy', fontsize=16)
    axs[0].set_ylabel('Accuracy', fontsize=14)
    axs[0].set_xlabel('Epoch', fontsize=14)
    axs[0].tick_params(axis='both', which='major', labelsize=12)
    axs[0].legend(['Train', 'Validation'], loc='lower right', fontsize=12)
    axs[0].grid(True, linestyle='--', alpha=0.6)

    # Plot training & validation loss values
    axs[1].plot(history.history['loss'], linewidth=2)
    axs[1].plot(history.history['val_loss'], linewidth=2)
    axs[1].set_title(f'{model_name} Model Loss', fontsize=16)
    axs[1].set_ylabel('Loss', fontsize=14)
    axs[1].set_xlabel('Epoch', fontsize=14)
    axs[1].tick_params(axis='both', which='major', labelsize=12)
    axs[1].legend(['Train', 'Validation'], loc='upper right', fontsize=12)
    axs[1].grid(True, linestyle='--', alpha=0.6)

    plt.tight_layout()
    fig.savefig(f'{model_name.lower().replace(" ", "_")}_training_history.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
# Plot the training history graphs
plot_model_history(lstm_history, "Improved Bidirectional LSTM")
plot_model_history(cnn_lstm_history, "Improved Bidirectional CNN-LSTM")

# Save models in multiple formats
model1.save('improved_lstm_model.keras')
model1.save('improved_lstm.keras')

model2.save('improved_cnn_lstm_model.keras')
model2.save('improved_cnn_lstm.keras')

In [ ]:
# Plot model comparison
plt.figure(figsize=(14, 8))
plt.subplot(1, 2, 1)
plt.plot(lstm_history.history['accuracy'], label='LSTM Train', linewidth=2, marker='o', markersize=4)
plt.plot(lstm_history.history['val_accuracy'], label='LSTM Validation', linewidth=2, marker='s', markersize=4)
plt.plot(cnn_lstm_history.history['accuracy'], label='CNN-LSTM Train', linewidth=2, marker='^', markersize=4)
plt.plot(cnn_lstm_history.history['val_accuracy'], label='CNN-LSTM Validation', linewidth=2, marker='d', markersize=4)
plt.title('Model Accuracy Comparison', fontsize=16)
plt.ylabel('Accuracy', fontsize=14)
plt.xlabel('Epoch', fontsize=14)
plt.tick_params(axis='both', which='major', labelsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(fontsize=12)

plt.subplot(1, 2, 2)
plt.plot(lstm_history.history['loss'], label='LSTM Train', linewidth=2, marker='o', markersize=4)
plt.plot(lstm_history.history['val_loss'], label='LSTM Validation', linewidth=2, marker='s', markersize=4)
plt.plot(cnn_lstm_history.history['loss'], label='CNN-LSTM Train', linewidth=2, marker='^', markersize=4)
plt.plot(cnn_lstm_history.history['val_loss'], label='CNN-LSTM Validation', linewidth=2, marker='d', markersize=4)
plt.title('Model Loss Comparison', fontsize=16)
plt.ylabel('Loss', fontsize=14)
plt.xlabel('Epoch', fontsize=14)
plt.tick_params(axis='both', which='major', labelsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(fontsize=12)

plt.tight_layout()
plt.savefig('model_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

# Model performance comparison
lstm_acc = score[1]
cnn_lstm_acc = model2.evaluate(X_test_cnn, Y_test)[1]

# Compare model performance with a bar chart
plt.figure(figsize=(10, 6))
models = ['Bidirectional LSTM', 'Bidirectional CNN-LSTM']
accuracies = [lstm_acc, cnn_lstm_acc]

# Create a gradient-colored bar chart
plt.bar(models, accuracies, color=['#3498db', '#2ecc71'], alpha=0.8, width=0.6)
plt.title('Model Accuracy Comparison', fontsize=16)
plt.ylabel('Test Accuracy', fontsize=14)
plt.ylim(0, 1.0)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Add value labels
for i, v in enumerate(accuracies):
    plt.text(i, v + 0.02, f'{v:.4f}', ha='center', fontsize=12, fontweight='bold')

plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.savefig('model_accuracy_comparison.png', dpi=300, bbox_inches='tight')
plt.show()
